In [ ]:
# chapter 4-1 IPythonの実行時間計測

In [3]:
n=100000

In [4]:
%timeit sum([1. / i**2 for i in range(1, n)])

10 loops, best of 3: 52.5 ms per loop


In [5]:
%%timeit s = 0.
for i in range(1, n):
    s += 1. / i**2

10 loops, best of 3: 49 ms per loop


In [6]:
import numpy as np

In [8]:
%timeit np.sum(1. / np.arange(1., n) ** 2)

1000 loops, best of 3: 567 µs per loop


In [9]:
# chapter 4-2 cProfileとIPythonによるコードプロファイル

In [11]:
import numpy as np
import matplotlib.pyplot as plt

In [12]:
%matplotlib inline

In [13]:
def step(*shape):
    # +1と-1の値をランダムに並べたn-vectorを作成
    return 2 * (np.random.random_sample(shape) < .5) -1

In [17]:
%%prun -s cumulative -q -l 10 -T prun0
n = 10000
iterations = 50
x = np.cumsum(step(iterations, n), axis=0)
bins = np.arange(-30, 30, 1)
y = np.vstack([np.histogram(x[i,:], bins)[0] for i in range(iterations)])

 
*** Profile printout saved to text file 'prun0'. 


In [18]:
print(open('prun0', 'r').read())

         3912 function calls in 0.071 seconds

   Ordered by: cumulative time
   List reduced from 47 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.071    0.071 {built-in method builtins.exec}
        1    0.000    0.000    0.071    0.071 <string>:2(<module>)
        1    0.004    0.004    0.046    0.046 <ipython-input-13-33db779c7ab4>:1(step)
        1    0.042    0.042    0.042    0.042 {method 'random_sample' of 'mtrand.RandomState' objects}
        1    0.000    0.000    0.021    0.021 <string>:6(<listcomp>)
       50    0.003    0.000    0.021    0.000 function_base.py:267(histogram)
       50    0.000    0.000    0.011    0.000 fromnumeric.py:717(sort)
       50    0.011    0.000    0.011    0.000 {method 'sort' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.004    0.004 fromnumeric.py:2067(cumsum)
        1    0.004    0.004    0.004    0.004 {method 'cumsum' of 'numpy.ndarray

In [19]:
%%prun -s cumulative -q -l 10 -T prun0
n = 10000
iterations = 500
x = np.cumsum(step(iterations, n), axis=0)
bins = np.arange(-30, 30, 1)
y = np.vstack([np.histogram(x[i,:], bins)[0] for i in range(iterations)])

 
*** Profile printout saved to text file 'prun0'. 


In [20]:
print(open('prun0', 'r').read())

         39012 function calls in 0.502 seconds

   Ordered by: cumulative time
   List reduced from 47 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.502    0.502 {built-in method builtins.exec}
        1    0.004    0.004    0.502    0.502 <string>:2(<module>)
        1    0.002    0.002    0.230    0.230 <string>:6(<listcomp>)
      500    0.024    0.000    0.228    0.000 function_base.py:267(histogram)
      500    0.001    0.000    0.140    0.000 fromnumeric.py:717(sort)
        1    0.052    0.052    0.140    0.140 <ipython-input-13-33db779c7ab4>:1(step)
      500    0.135    0.000    0.135    0.000 {method 'sort' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.126    0.126 fromnumeric.py:2067(cumsum)
        1    0.126    0.126    0.126    0.126 {method 'cumsum' of 'numpy.ndarray' objects}
        1    0.087    0.087    0.087    0.087 {method 'random_sample' of 'mtrand.RandomStat

In [21]:
# 4-3 line_profilerを使った行単位のコードプロファイル

In [22]:
# line_profilerのインストール
# !pip install line_profiler

    100% |████████████████████████████████| 71kB 1.0MB/s ta 0:00:011
  Running setup.py bdist_wheel for line-profiler ... - \ | / - \ done
  Stored in directory: /Users/Really/Library/Caches/pip/wheels/a8/aa/3b/1d4570cc467313caec92c9d51e6543b598b835ab95cfe9c726
Successfully built line-profiler


In [23]:
import numpy as np

In [25]:
%load_ext line_profiler

In [39]:
%%writefile simulation.py
import numpy as np
def step(*shape):
    # +1と-1の値をランダムに並べたn-vectorを作成
    return 2 * (np.random.random_sample(shape) < .5) -1
def simulate(iterations, n = 10000):
    s = step(iterations, n)
    x = np.cumsum(s, axis=0)
    bins = np.arange(-30, 30, 1)
    y = np.vstack([np.histogram(x[i,:], bins)[0] for i in range(iterations)])
    return y

Overwriting simulation.py


In [40]:
import simulation

In [41]:
%lprun -T lprof0 -f simulation.simulate simulation.simulate(50)


*** Profile printout saved to text file 'lprof0'. 


In [42]:
print(open('lprof0', 'r').read())

Timer unit: 1e-06 s

Total time: 0.03854 s
File: /Users/Really/_IPython_Notebook/simulation.py
Function: simulate at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
     5                                           def simulate(iterations, n = 10000):
     6         1         9694   9694.0     25.2      s = step(iterations, n)
     7         1         2264   2264.0      5.9      x = np.cumsum(s, axis=0)
     8         1           27     27.0      0.1      bins = np.arange(-30, 30, 1)
     9         1            5      5.0      0.0      y = np.vstack([np.histogram(x[i,:], bins)[0] for i in range(iterations)])
    10         1        26548  26548.0     68.9      return y


In [43]:
%lprun -T lprof0 -f simulation.simulate simulation.simulate(500)


*** Profile printout saved to text file 'lprof0'. 


In [44]:
print(open('lprof0', 'r').read())

Timer unit: 1e-06 s

Total time: 0.523659 s
File: /Users/Really/_IPython_Notebook/simulation.py
Function: simulate at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
     5                                           def simulate(iterations, n = 10000):
     6         1       117290 117290.0     22.4      s = step(iterations, n)
     7         1       145507 145507.0     27.8      x = np.cumsum(s, axis=0)
     8         1           22     22.0      0.0      bins = np.arange(-30, 30, 1)
     9         1            3      3.0      0.0      y = np.vstack([np.histogram(x[i,:], bins)[0] for i in range(iterations)])
    10         1       260835 260835.0     49.8      return y


In [45]:
# chapter 4-4 memory_profilerを使ったメモリ使用状況のプロファイル

In [4]:
# memory_profilerのインストール
# !pip install memory_profiler

  Running setup.py bdist_wheel for memory-profiler ... - \ done
  Stored in directory: /Users/Really/Library/Caches/pip/wheels/fe/04/c4/0f3a20494cedaeca77559759d98717b036152a6b0e86e917b4
Successfully built memory-profiler


In [1]:
import simulation

In [5]:
%load_ext memory_profiler

In [7]:
%mprun -T mprof0 -f simulation.simulate simulation.simulate(50)



*** Profile printout saved to text file mprof0. 


In [8]:
%mprun -T mprof0 -f simulation.simulate simulation.simulate(500)



*** Profile printout saved to text file mprof0. 


In [9]:
# chapter 4-5 不必要な配列コピーを削除するためのNumPy内部構造解説

In [12]:
import numpy as np

In [ ]:
a = np.zeros(10)

In [10]:
def id(x):
# 配列のメモリブロックのアドレスを返す
    return x.__array_interface__['data'][0]

In [23]:
id(a), id(a[1:])
# オフセットが異なればメモリ位置も異なる

(140329807768096, 140329807768104)

In [16]:
# 2つの配列が同じバッファを共有しているのかを確かめるためのより汎用で厳密な方法

def get_data_base(arr):
    # 指定されたNumPy配列のデータを格納している配列のアドレスを求める
    base = arr
    while isinstance(base.base, np.ndarray):
        base = base.base
        
    return base

def arrays_share_data(x, y):
    return get_data_base(x) is get_data_base(y)

In [24]:
print(arrays_share_data(a,a.copy()),
     arrays_share_data(a,a[1:]))

False True


In [25]:
a = np.zeros(10); aid = id(a); aid

140329841243024

In [26]:
b = a.copy(); id(b) == aid
# 違うアドレスになる

False

In [29]:
a *= 2; id(a) == aid
# 同じアドレス、元の配列の内容が変更される

True

In [31]:
a = a*2; id(a) == aid
# 違うアドレス、元の配列が暗黙のうちにコピーされ新しい配列が作成される

False

In [34]:
%%timeit a = np.zeros(10000000)
a *= 2
# 変更、速い

The slowest run took 4.72 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 3: 8.51 ms per loop


In [35]:
%%timeit a = np.zeros(10000000)
a = a*2
# コピー、遅い

10 loops, best of 3: 29.7 ms per loop


In [36]:
a = np.zeros((10, 10)); aid = id(a)

In [38]:
b = a.reshape((1, -1)); id(b) == aid
# 2次元配列の形状変更ではコピーが起こらないが、、

True

In [39]:
c = a.T.reshape((1, -1)); id(c) == aid
# 転置を伴う場合にはコピーが発生する

False

In [44]:
d = a.flatten(); id(d) == aid
# flattenは常に配列のコピーを返すが、、

False

In [45]:
e = a.ravel(); id(e) == aid
# ravelは必要な場合にだけコピーを行う

True

In [46]:
%timeit a.flatten()

The slowest run took 22.88 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 915 ns per loop


In [47]:
%timeit a.ravel()

The slowest run took 26.77 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 284 ns per loop


In [ ]:
# ブロードキャストルール(2つの配列の形状が全く同じか、いずれかの次元長が1である場合に両者が互換であるとする)
# により配列の形状が異なっていても直接計算できる。
# reshapeやtileを使って形状を合わせる必要は必ずしもない。

In [48]:
n = 1000

In [50]:
a = np.arange(n)
ac = a[:, np.newaxis] # 列ベクトル
ar = a[np.newaxis, :] # 行ベクトル

In [53]:
% timeit np.tile(ac, (1, n)) * np.tile(ar, (n, 1))
# タイリングを使って計算、遅い

100 loops, best of 3: 13.8 ms per loop


In [54]:
%timeit ac * ar
# ブロードキャストにより直接計算、速い

100 loops, best of 3: 3.29 ms per loop


In [ ]:
# 以下の2つの例のパフォーマンスの違いの理由は？
# 答えは書いてない。NumPyの内部構造が行優先だから？？？

In [56]:
a = np.random.rand(5000, 5000)

In [59]:
%timeit a[0,:].sum()
# 速い

The slowest run took 6.51 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 7.92 µs per loop


In [60]:
%timeit a[:,0].sum()
# 遅い

10000 loops, best of 3: 77.5 µs per loop


In [ ]:
# chapter 4-6 NumPyのストライドトリック